In [1]:
with open("./text_data/input.txt", "r", encoding="utf-8") as f:
    text = f.read()

print(f"Length of the text: {len(text)}")

Length of the text: 1125354


In [2]:
import torch

In [3]:
chars = sorted(list(set(text)))
print(f"The list of all unique characters: {chars}")

vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

The list of all unique characters: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
string_to_index = {ch: i for i, ch in enumerate(chars)}
index_to_string = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_index[c] for c in s]
decode = lambda l: ''.join(index_to_string[i] for i in l)

print(string_to_index)

print("\n\n")
print(encode("Hello world!"))
print(decode(encode("Hello world!")))

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}



[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
Hello world!


In [5]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")
print(enc.n_vocab)
print(enc.encode("Hi There"))

50257
[17250, 1318]


In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1125354]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
n = int(0.9 * len(data))
train_data = data[:n]
valid_data = data[n:]

In [8]:
block_size = 8
print(train_data[:block_size + 1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


In [9]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
print(f"\n\nx: {x}")
print(f"y: {y}")
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"When input is {context} the target is {target}")



x: tensor([18, 47, 56, 57, 58,  1, 15, 47])
y: tensor([47, 56, 57, 58,  1, 15, 47, 58])
When input is tensor([18]) the target is 47
When input is tensor([18, 47]) the target is 56
When input is tensor([18, 47, 56]) the target is 57
When input is tensor([18, 47, 56, 57]) the target is 58
When input is tensor([18, 47, 56, 57, 58]) the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [10]:
torch.manual_seed(1234)
batch_size = 4
block_size = 8


def get_batch(split):
    data = train_data if split == "train" else valid_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])

    return x, y

In [11]:
xb, yb = get_batch("train")
print(f"Inputs: {xb.shape}\n, {xb}\n")
print(f"Targets: {yb.shape}\n, {yb}")
print("-" * 50)

tensor([168055, 420071, 144026, 164285])
Inputs: torch.Size([4, 8])
, tensor([[47, 57,  1, 39,  1, 55, 59, 39],
        [39, 52,  1, 47, 58,  1, 47, 57],
        [58,  1, 61, 47, 58, 46,  1, 63],
        [42,  1, 42, 61, 43, 50, 50, 57]])

Targets: torch.Size([4, 8])
, tensor([[57,  1, 39,  1, 55, 59, 39, 56],
        [52,  1, 47, 58,  1, 47, 57,  1],
        [ 1, 61, 47, 58, 46,  1, 63, 53],
        [ 1, 42, 61, 43, 50, 50, 57, 11]])
--------------------------------------------------


In [12]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f"When input is {context} the target is {target}")

When input is tensor([47]) the target is 57
When input is tensor([47, 57]) the target is 1
When input is tensor([47, 57,  1]) the target is 39
When input is tensor([47, 57,  1, 39]) the target is 1
When input is tensor([47, 57,  1, 39,  1]) the target is 55
When input is tensor([47, 57,  1, 39,  1, 55]) the target is 59
When input is tensor([47, 57,  1, 39,  1, 55, 59]) the target is 39
When input is tensor([47, 57,  1, 39,  1, 55, 59, 39]) the target is 56
When input is tensor([39]) the target is 52
When input is tensor([39, 52]) the target is 1
When input is tensor([39, 52,  1]) the target is 47
When input is tensor([39, 52,  1, 47]) the target is 58
When input is tensor([39, 52,  1, 47, 58]) the target is 1
When input is tensor([39, 52,  1, 47, 58,  1]) the target is 47
When input is tensor([39, 52,  1, 47, 58,  1, 47]) the target is 57
When input is tensor([39, 52,  1, 47, 58,  1, 47, 57]) the target is 1
When input is tensor([58]) the target is 1
When input is tensor([58,  1]) the

In [13]:
import torch.nn as nn
from torch.nn import functional as F

In [25]:
torch.manual_seed(1234)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        print(self.token_embedding_table)

    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx)  # (B=4, T=8, C=65)

        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=1)
            return idx

In [26]:
m = BigramLanguageModel(vocab_size=vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss)

Embedding(65, 65)
torch.Size([32, 65])
tensor(4.6862, grad_fn=<NllLossBackward0>)
